In [32]:
import cv2
import numpy as np
import pandas as pd
import requests
from PIL import Image
from io import BytesIO
from tqdm import tqdm
import itertools
import boto3
import base64

In [2]:
def show(img, offset=1):
    shape = img.shape
#     offset = 1
    s = 0
    if shape[0] > 1000:
        s = int(10/(int(str(shape[0])[0])+offset))*0.1
    elif shape[1] > 1000:
        s = int(10/(int(str(shape[1])[0])+offset))*0.1
    if s:
        img = cv2.resize(img, None, fx=s, fy=s)
    cv2.imshow("Img", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [3]:
base = cv2.imread('2.jpg')

In [16]:
df = pd.read_csv('product_export_2022-05-28-07-26-33.csv')

In [42]:
def read_image_from_url(url):
    img = Image.open(BytesIO(requests.get(url).content))
    im_arr = np.array(img)
    im_processed = cv2.cvtColor(im_arr, cv2.COLOR_RGB2BGR)
    return im_processed


def apply_brightness_contrast(input_img, brightness = 0, contrast = 0):
    
    if brightness != 0:
        if brightness > 0:
            shadow = brightness
            highlight = 255
        else:
            shadow = 0
            highlight = 255 + brightness
        alpha_b = (highlight - shadow)/255
        gamma_b = shadow
        
        buf = cv2.addWeighted(input_img, alpha_b, input_img, 0, gamma_b)
    else:
        buf = input_img.copy()
    
    if contrast != 0:
        f = 131*(contrast + 127)/(127*(131-contrast))
        alpha_c = f
        gamma_c = 127*(1-f)
        
        buf = cv2.addWeighted(buf, alpha_c, buf, 0, gamma_c)

    return buf

def convert_mockup(base, resized_book):
    mask = np.ones_like(base)
    k = apply_brightness_contrast(resized_book, 70, 0)
    insert_img = k.copy()

    x_offset=1007  
    y_offset=304

    x_end = x_offset+resized_book.shape[1]
    y_end = y_offset+resized_book.shape[0]
    mask[y_offset:y_end, x_offset:x_end] = insert_img 

    base[y_offset:y_end, x_offset:x_end] = insert_img 
    return base


def upload_s3(img, filename):

    #Creating Session With Boto3.
    session = boto3.Session(
    aws_access_key_id='A',
    aws_secret_access_key='A'
    )

    #Creating S3 Resource From the Session.
    s3 = session.resource('s3')


    s3_object = s3.Object('lm-staging-test', filename)
    img_str = cv2.imencode('.jpg', mockup)[1].tostring()
    result = s3_object.put(Body=img_str, ContentType= 'image/jpg')
    
    return f"https://lm-staging-test.s3.ap-south-1.amazonaws.com/{filename}"

In [18]:
urls = {}
for index, i in tqdm(enumerate(df['images'])):
    try:
        url = i.split('!')[0].strip()
        urls[index]= url
    except Exception as e:
        print(f"{i} -- {e}")


2924it [00:00, 144721.63it/s]

nan -- 'float' object has no attribute 'split'
nan -- 'float' object has no attribute 'split'
nan -- 'float' object has no attribute 'split'
nan -- 'float' object has no attribute 'split'
nan -- 'float' object has no attribute 'split'
nan -- 'float' object has no attribute 'split'
nan -- 'float' object has no attribute 'split'
nan -- 'float' object has no attribute 'split'


In [9]:
sliced_url_dict = dict(itertools.islice(urls.items(), 10))  # slice dictionary to n items

In [48]:
for index in sliced_url_dict:
    img_url = sliced_url_dict[index]
    book = read_image_from_url(img_url) 
    file_name = img_url.split('/')[-1]
    resized_book = cv2.resize(book, (984, 1372))  # resize to mockup book background
    mockup = convert_mockup(base, resized_book)
    s3_link = upload_s3(mockup, file_name)
    df['images'][index] = df['images'][index].replace(img_url, s3_link)

/home/sourav/anaconda3/envs/dsenv/lib/python3.7/site-packages/ipykernel_launcher.py:62: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
/home/sourav/anaconda3/envs/dsenv/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


'Swadesh-Savata-o-bishwa-by-jiban-Mukhapadhai.jpg'

In [50]:
df['images'][0]

'https://lm-staging-test.s3.ap-south-1.amazonaws.com/lifes-amazing-secrets-how-to-find-balance-and-purpose-in-your-life.jpg ! alt :  ! title :  ! desc :  ! caption : '

In [51]:
df.to_csv("prod.csv")

In [53]:
df

,post_title,post_name,post_parent,ID,post_content,post_excerpt,post_status,post_password,menu_order,post_date,...,meta:_yoast_wpseo_canonical,meta:_yoast_wpseo_bctitle,meta:_yoast_wpseo_meta-robots-adv,meta:_yoast_wpseo_is_cornerstone,meta:_yoast_wpseo_metadesc,meta:_yoast_wpseo_linkdex,meta:_yoast_wpseo_estimated-reading-time-minutes,meta:_yoast_wpseo_content_score,meta:_yoast_wpseo_title,meta:_yoast_wpseo_metakeywords
0,Life's Amazing Secrets: How to Find Balance an...,lifes-amazing-secrets-how-to-find-balance-and-...,NaN,2241,"<h2 class=""books"">Life's Amazing Secrets: How ...","<img class=""alignnone size-medium wp-image-178...",publish,NaN,0,2018-10-23 03:54:56,...,NaN,NaN,NaN,NaN,Life's Amazing Secrets: How to Find Balance an...,47.0,1.0,30.0,%%title%%,NaN
1,Swadesh Savata o bishwa by jiban Mukhapadhai (...,swadesh-savyata-o-bishwa-by-jiban-mukhapadhay-...,NaN,2323,<strong>Swadesh Savyata O Bishwa By Jiban Mukh...,"<img class=""alignnone size-medium wp-image-178...",publish,NaN,0,2018-11-09 00:32:47,...,NaN,NaN,NaN,NaN,Swadesh Savyata O Bishwa By Jiban Mukhopadhyay...,41.0,1.0,60.0,Swadesh Savyata O Bishwa By Jiban Mukhopadhyay...,NaN
2,Nimashes Math by Chanchol Ghosh Book Vol 1,nimashe-math-by-chanchal-ghosh-book-vol-1,NaN,2328,<strong>Nimashe Math by Chanchal Ghosh Book-Vo...,"<img class=""alignnone size-medium wp-image-178...",publish,NaN,0,2018-11-09 01:22:20,...,NaN,NaN,NaN,NaN,Nimashe Math by Chanchal Ghosh Book-Vol 1 | Au...,44.0,1.0,90.0,NaN,NaN
3,Objective Arithmetic SSC and Railway Exam Special,objective-arithmetic-ssc-and-railway-exam-spec...,NaN,2333,<strong>Objective Arithmetic SSC and Railway E...,"<img class=""alignnone size-medium wp-image-178...",publish,NaN,0,2018-11-09 01:34:40,...,NaN,NaN,NaN,NaN,Objective Arithmetic SSC and Railway Exam Spec...,45.0,1.0,60.0,%%title%% %%page%%,NaN
4,Bharot o Pachimbangar Bhugol By Kartick Chandr...,bharot-o-pachimbangar-bhugol-by-kartick-chandr...,NaN,2341,<p>Bharot o Pachimbangar Bhugol By <em>Kartick...,"<img class=""alignnone size-medium wp-image-178...",publish,NaN,0,2018-11-11 00:32:40,...,NaN,NaN,NaN,NaN,Bharot o Pachimbangar Bhugol By Kartick Chandr...,54.0,1.0,90.0,Bharot o Pachimbangar Bhugol By Kartick Chandr...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2919,SLST Geography By Kanailal Mandol &amp; Alokna...,slst-geography-by-kanailal-mandol-aloknath-ghosh,NaN,180692,"স্নাতক ও স্নাতকোঙার কোর্স এবং SSC, NET SET &am...",SLST Geography By Kanailal Mandol &amp; Alokna...,publish,NaN,0,2022-05-20 17:02:28,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2920,TAPATIR SLST Botany Encyclopedia Graduate (9&a...,tapatir-slst-botany-encyclopedia-graduate-910,NaN,180694,TAPATIR SLST Botany Encyclopedia Graduate (9&a...,TAPATIR SLST Botany Encyclopedia Graduate (9&a...,publish,NaN,0,2022-05-20 17:09:55,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2921,TAPATIR SLST Zoology ENCYCLOPEDIA Post Graduat...,tapatir-slst-zoology-encyclopedia-post-graduat...,NaN,180697,TAPATIR SLST Zoology ENCYCLOPEDIA Post Graduat...,TAPATIR SLST Zoology ENCYCLOPEDIA Post Graduat...,publish,NaN,0,2022-05-20 17:14:17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2922,TAPATIR SLST Sanskrit ENCYCLOPEDIA Post Gradua...,tapatir-slst-sanskrit-encyclopedia-post-gradua...,NaN,180699,TAPATIR SLST Sanskrit ENCYCLOPEDIA Post Gradua...,TAPATIR SLST Sanskrit ENCYCLOPEDIA Post Gradua...,publish,NaN,0,2022-05-20 17:18:09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
